In [1]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os

## Data Loading

### Writing PyG Dataset

In [2]:
import torch

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [4]:
from torch_geometric.utils import from_networkx, to_networkx

In [5]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 2)
    ind = 0
    for node in G.nodes():
        x[ind][0] = G.degree[node]
        ind += 1
    return x

In [6]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 1)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 64)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [7]:
DATA_PATH = "../data-2d/data/low_decomp_sahil_dataset.pkl.gz"

In [8]:
laman_data = LamanDataset("", DATA_PATH)

Processing...
Done!


## Split into Train / Test

In [9]:
from torch.utils.data import random_split

proportions = [.5, .5]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [10]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 512, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 512, shuffle=True)

In [11]:
print("Number of train batches: ", len(train_loader))
print("Number of train batches: ", len(test_loader))

Number of train batches:  32
Number of train batches:  8


In [12]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 72974], x=[19919, 2], label=[512], num_nodes=19919, batch=[19919], ptr=[513])


## Model Architecture & Training

In [13]:
from basic_gcn.gcn import GCN

In [14]:
model = GCN(2)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(2, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Number of parameters:  12801


In [15]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [16]:
def train(data):
    ind = 0
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x, batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, None

In [17]:
def check_accuracy(model, loader):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x, batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)*100

In [18]:
print("Starting training...")
losses = []

for epoch in range(10):
    loss, h = train(train_loader)
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader), check_accuracy(model, test_loader)
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
    

Starting training...
Epoch 0 | Train loss 0.40619194507598877
Train Accuracy 80.7 | Test Accuracy 80.375
Epoch 1 | Train loss 0.2201772779226303
Train Accuracy 87.69375000000001 | Test Accuracy 87.75
Epoch 2 | Train loss 0.21943190693855286
Train Accuracy 89.5625 | Test Accuracy 89.7
Epoch 3 | Train loss 0.2736218273639679
Train Accuracy 89.60000000000001 | Test Accuracy 89.97500000000001
Epoch 4 | Train loss 0.2325899302959442
Train Accuracy 91.61875 | Test Accuracy 92.35
Epoch 5 | Train loss 0.18732032179832458
Train Accuracy 91.71875 | Test Accuracy 92.225
Epoch 6 | Train loss 0.16231980919837952
Train Accuracy 90.19375 | Test Accuracy 90.14999999999999
Epoch 7 | Train loss 0.29242974519729614
Train Accuracy 92.975 | Test Accuracy 93.175
Epoch 8 | Train loss 0.12299274653196335
Train Accuracy 91.90625 | Test Accuracy 92.4
Epoch 9 | Train loss 0.1566777527332306
Train Accuracy 91.53750000000001 | Test Accuracy 92.175


In [19]:
# sanity check

In [ ]:
# square
import networkx as nx
square = nx.Graph()
square.add_edge(0, 1)
square.add_edge(1, 3)
square.add_edge(0, 2)
square.add_edge(2, 3)

In [3]:
# square with cross bar (rigid)
import networkx as nx
square_bar = nx.Graph()
square_bar.add_edge(0, 1)
square_bar.add_edge(1, 3)
square_bar.add_edge(0, 2)
square_bar.add_edge(2, 3)
square_bar.add_edge(0, 3)

In [ ]:
# triangle
import networkx as nx
triangle = nx.Graph()
triangle.add_edge(0, 1)
triangle.add_edge(0, 2)
triangle.add_edge(1, 2)

In [ ]:
# pentagon
import networkx as nx
pentagon = nx.Graph()
pentagon.add_edge(0, 1)
pentagon.add_edge(1, 3)
pentagon.add_edge(3, 4)
pentagon.add_edge(4, 2)
pentagon.add_edge(2, 0)

In [1]:
graph_as_data = from_networkx(pentagon)
graph_as_data.x = generate_feature_vector(pentagon)
graph_as_data.label = 1

NameError: name 'from_networkx' is not defined

In [2]:
graph_as_data

NameError: name 'graph_as_data' is not defined

In [ ]:
print(x)

In [ ]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [ ]:
for batch in validation_set:
    pred = model(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])